<a href="https://colab.research.google.com/github/ShivikaPrasanna/rag-llm-starter/blob/main/starter_rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-f0c158413defd454.parquet', 'test': 'data/test-00000-of-00001-06d83c58a8ea10e8.parquet'}
df = pd.read_parquet("hf://datasets/neural-bridge/rag-dataset-1200/" + splits["train"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df

,context,question,answer
0,Francisco Rogers found the answer to a search ...,Who found the answer to a search query collar ...,Francisco Rogers found the answer to a search ...
1,"A few nights ago, my husband and I were watchi...",What are some of the potential negative impact...,The context discusses that charity can sometim...
2,Game Notes EDM vs BUF\nBuffalo Sabres (Head Co...,Who were the three stars in the NHL game betwe...,"The three stars were Ryan O’Reilly, Brian Gion..."
3,"Find Movers in Santa Clarita, CA 91390 - Movin...",What services does Pearl Moving Company in San...,"Pearl Moving Company Santa Clarita, 91390 offe..."
4,"Senior Planning Engineer in England - London, ...",What are the responsibilities of a Senior Plan...,The responsibilities of a Senior Planning Engi...
...,...,...,...
955,"I grew up in Charleston, West Virginia, went t...",Where did Anthony Spencer receive his degree i...,Anthony Spencer received his degree in marketi...
956,Cool Chair Covers\nMarvelous Cool Chair Covers...,What is the purpose of the Cool Chair Covers p...,The Cool Chair Covers photo collection is mean...
957,SO0561 : Old and new in Llandrindod Wells\ntak...,What is the newest entrant onto the British Hi...,The newest entrant onto the British High Stree...
958,"$-1468 2Beds 1Baths 1,068Sq Ft 16832 S Fair RD...",What are the features of the property located ...,"The property at 16832 S Fair RD Kinross, MI 49..."


In [ ]:
context = df['context'].tolist()
question = df['question'].tolist()
answer = df['answer'].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModel
import faiss, torch
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
tokenized = tokenizer(context, padding=True, truncation=True, return_tensors='pt', max_length=512)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_embeddings(documents, batch_size=32):
    embeddings = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        tokenized = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)

        with torch.no_grad():
            batch_embeddings = model(**tokenized).last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(batch_embeddings)

    return np.vstack(embeddings)

# Compute embeddings in batches
embeddings = compute_embeddings(context)

# with torch.no_grad():
#   embeddings = model(**tokenized).last_hidden_state.mean(dim=1).numpy()

In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
import openai

openai.api_key = ''

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def rag_generator(query):
    query_tokens = tokenizer(query, return_tensors='pt', max_length=512).to(device)
    with torch.no_grad():
        query_embedding = model(**query_tokens).last_hidden_state.mean(dim=1).cpu().numpy()

    D, I = index.search(query_embedding, k=5)

    retrieved_context = "\n".join([context[idx] for idx in I[0]])

    return retrieved_context

In [ ]:
def generate_response(prompt):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Your task is to answer the question as precisely as possible."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    return response.choices[0].message.content

query = "What is the person seeking in the context?"
retrieved_docs = rag_generator(query)
prompt = f"Question: {query}\n\nContext:\n{retrieved_docs}\n\nAnswer:"

response_text = generate_response(prompt)
print(response_text)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The person is seeking guidance or advice on how to balance their emotions and feelings with their worship and obedience to Jesus, and avoid the pitfalls of emotionalism and satisfaction-centered worship.
